In [ ]:
 class MyDataset_VS(Dataset):
        def __init__(self, root, df_tr=None, df_te=None, df_va=None, df_te2=None, protein_dict=None, proteinMode=0, compoundMode=1,
                    pocket_radius=20, shake_nodes=None, 
                    transform=None, pre_transform=None, pre_filter=None, generate_3D_conf = False,
                    protein_res_id_dict=None, nci_df=None, ligand_atom_id_dict=None, cfg_mode=None,
                    right_pocket_by_distance=True,
                    ):
            self.trainindx = 0
            self.testindx = 0
            self.valindx = 0
            self.test2indx = 0
            self.test_df = df_te
            self.train_df = df_tr
            self.val_df = df_va
            self.test2_df = df_te2
            self.testindx = len(self.test_df) 
            self.expected_test_count = len(self.test_df) 
            self.trainindx = len(self.train_df) +  self.testindx 
            self.expected_train_count = len(self.train_df) 
            self.valindx = len(self.val_df) + self.trainindx
            self.expected_val_count = len(self.val_df)
            self.test2indx = len(self.test2_df) + self.valindx 
            self.expected_test2_count = len(self.test2_df)
            self.protein_dict = protein_dict
            super().__init__(root, transform, pre_transform, pre_filter)
            print(self.processed_paths)
            self.data = torch.load(self.processed_paths[0])
            self.protein_dict = torch.load(self.processed_paths[1])
            self.nci_df=nci_df
            self.protein_res_id_dict = protein_res_id_dict
            self.ligand_atom_id_dict = ligand_atom_id_dict
            self.proteinMode = proteinMode
            self.pocket_radius = pocket_radius
            self.compoundMode = compoundMode
            self.shake_nodes = shake_nodes
            self.generate_3D_conf = generate_3D_conf
            self.cfg_mode = cfg_mode if cfg_mode else "tankbind"
            self.right_pocket_by_distance=right_pocket_by_distance
        @property
        def processed_file_names(self):
            return ['data.pt', 'protein.pt']

        def process(self):
            _data_total = pd.concat([self.test_df, self.train_df, self.val_df, self.test2_df]) 
            torch.save(_data_total, self.processed_paths[0])
            torch.save(self.protein_dict, self.processed_paths[1])

        def len(self):
            return self.expected_test_count + self.expected_train_count \
                + self.expected_val_count + self.expected_test2_count

        def get_idx_split(self):
            return self.expected_test_count, \
                self.expected_test_count + self.expected_train_count, \
                self.expected_test_count + self.expected_train_count + self.expected_val_count, \
                self.expected_test_count + self.expected_train_count + self.expected_val_count + self.expected_test2_count

        def get(self, idx):
            line = self.data.iloc[idx]
            canonique_smiles = line['canonique_smiles']
            pocket_com = line['pocket_com']
            pocket_com = np.array(pocket_com.split(",")).astype(float) if type(pocket_com) == str else pocket_com
            pocket_com = pocket_com.reshape((1, 3))
            use_whole_protein = line['use_whole_protein'] if "use_whole_protein" in line.index else False
            protein_name = line['pdb_code']
            pocket_name = line['pocket_name']
            protein_node_xyz, protein_seq, protein_node_s, protein_node_v, protein_edge_index, protein_edge_s, protein_edge_v = self.protein_dict[protein_name]
            ligand_fpath = line['ligand_fpath']
            ligand_ftype = line['ligand_ftype']
            ligand_name = line['ligand_name']
            affinity = line['affinity']
            if self.right_pocket_by_distance:
                right_pocket_by_distance = line['right_pocket_by_distance']
            if ligand_ftype == ".pdb":
                mol = Chem.MolFromPDBFile(ligand_fpath)
            elif ligand_ftype == ".sdf":
                mol = Chem.MolFromMolFile(ligand_fpath)
            mol.Compute2DCoords()  
            if self.cfg_mode == "tankbind":
                try:
                    coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list, pair_dis_distribution = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)
                except Exception as e:
                    return protein_name+" ERROR_II : "+str(e)
                try:
                    data, input_node_list, keepNode = construct_data_from_graph_gvp(protein_node_xyz, protein_seq, protein_node_s, 
                                        protein_node_v, protein_edge_index, protein_edge_s, protein_edge_v,
                                        coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list,
                                        pocket_radius=self.pocket_radius, use_whole_protein=use_whole_protein, includeDisMap=True,
                                        use_compound_com_as_pocket=False, chosen_pocket_com=pocket_com, compoundMode=self.compoundMode)
                    data.compound_pair = pair_dis_distribution.reshape(-1, 16)
                except Exception as e:
                    return protein_name+" ERROR_III : "+str(e)
                try:
                    data.right_pocket_by_distance = right_pocket_by_distance
                    data.affinity = affinity
                    data.dataname = protein_name + "_" + ligand_name + "_" + pocket_name
                except Exception as e:
                    return protein_name+" ERROR_IV : "+str(e)
            elif self.cfg_mode == "nciyes":
                protein_res_ids = self.protein_res_id_dict[protein_name]
                if (len(protein_res_ids.keys())-1) != protein_res_ids[list(protein_res_ids.keys())[-1]]:
                    return protein_name+" ERROR_I : protein_res_ids length error."
                try:
                    coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list, pair_dis_distribution = sx_extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True, generate_3D_conf=False)
                except Exception as e:
                    return protein_name+" ERROR_II : "+str(e)
                    # y is distance map, instead of contact map.
                try:
                    data, input_node_list, keepNode = sx_construct_data_from_graph_gvp(protein_node_xyz, protein_seq, protein_node_s, 
                                        protein_node_v, protein_edge_index, protein_edge_s, protein_edge_v,
                                        coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list,
                                        pocket_radius=self.pocket_radius, use_whole_protein=use_whole_protein, includeDisMap=True,
                                        use_compound_com_as_pocket=False, chosen_pocket_com=pocket_com, compoundMode=self.compoundMode)
                except Exception as e:
                    return protein_name+" ERROR_III : "+str(e)
                data.compound_pair = pair_dis_distribution.reshape(-1, 16)
                kept_res_ids = [_id for (_id, _keep) in zip(protein_res_ids, keepNode) if _keep]

                try:
                    atom_ids = self.ligand_atom_id_dict[protein_name]
                    #data.nci_sequence = torch.Tensor(sx_get_nci_matrix_by_dict(protein_name, ligand_name, res_full_id, atom_ids, self.nci_df).flatten())
                    data.nci_sequence = torch.tensor(sx_get_nci_matrix_by_dict(protein_name, ligand_name, kept_res_ids, atom_ids, self.nci_df).flatten())
                    data.pair_shape = (len(kept_res_ids), len(atom_ids))
                    data.right_pocket_by_distance = right_pocket_by_distance
                    data.affinity = affinity
                    data.dataname = protein_name + "_" + ligand_name + "_" + pocket_name
                except Exception as e:
                    return protein_name+" ERROR_IV : "+str(e)
            elif self.cfg_mode == "frag":
                try:
                    coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list, pair_dis_distribution = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)
                except Exception as e:
                    return protein_name+" ERROR_II : "+str(e)
                try:
                    data, input_node_list, keepNode = construct_data_from_graph_gvp(protein_node_xyz, protein_seq, protein_node_s, 
                                        protein_node_v, protein_edge_index, protein_edge_s, protein_edge_v,
                                        coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list,
                                        pocket_radius=self.pocket_radius, use_whole_protein=use_whole_protein, includeDisMap=True,
                                        use_compound_com_as_pocket=False, chosen_pocket_com=pocket_com, compoundMode=self.compoundMode)
                    data.compound_pair = pair_dis_distribution.reshape(-1, 16)
                except Exception as e:
                    return protein_name+" ERROR_III : "+str(e)
                try:
                    data.right_pocket_by_distance = right_pocket_by_distance
                    data.affinity = affinity
                    data.dataname = protein_name + "_" + ligand_name + "_" + pocket_name
                except Exception as e:
                    return protein_name+" ERROR_IV : "+str(e)
                #TODO: write your codes here. Refer to "if self.cfg_mode=="tankband" above.

            return data

In [ ]:
cfg_use_saved_files = False # If you have already generated all the data required.
cfg_save_files = True # If you want to save generated data.

In [2]:
#time
if True:
    qrint("\n", r=False)
    qrint(f"{R}dataset{S} generation or load")
    qrint(saveconfig(cfg_use_saved_files, cfg_save_files))
    qrint("\n", r=False)
    qrint(f"{R}info split{S} for dataset processing")


    if cfg_timesplit: # cfg_timesplit = True
        qrint(f"{R}Timesplit{S}: for dataset processing, the results will always be saved.")
        qrint(f"Checking timesplit files：")
        if not os.path.exists(f"{save_files_path}timesplit_train_no_lig_overlap.txt") or not os.path.exists(f"{save_files_path}timesplit_val_no_lig_overlap.txt") or not os.path.exists(f"{save_files_path}timesplit_test.txt"):
            qrint(f"Timesplit files not found. {R}Length split strategy{S} will be applied.")
            cfg_timesplit = False
        else:
            qrint(f"Found timesplit files. {R}Timesplit{S} will be applied.")
            with open(f"{save_files_path}timesplit_train_no_lig_overlap.txt", "r") as f:
                _train = [_t.replace("\n", "") for _t in f.readlines()]
            with open(f"{save_files_path}timesplit_val_no_lig_overlap.txt", "r") as f:
                _val = [_t.replace("\n", "") for _t in f.readlines()]
            with open(f"{save_files_path}timesplit_test.txt", "r") as f:
                _test = [_t.replace("\n", "") for _t in f.readlines()]
                
            dataset_path = f"{save_files_path}dataset/"
            qrint(f"Processing {R}dataset{S}:")
            if not cfg_use_saved_files:
                os.system(f"rm -r {dataset_path}")
                for _s in ["data"]:
                    os.system(f"mkdir -p {dataset_path}{_s}")
            info_test2 = info[~(info.pdb_code.isin(_train)|info.pdb_code.isin(_val)|info.pdb_code.isin(_test))]
            _total_set = MyDataset_VS(f"{dataset_path}data/", df_tr=info[info.pdb_code.isin(_train)],df_te=info[info.pdb_code.isin(_test)],df_va=info[info.pdb_code.isin(_val)], df_te2=info_test2, \
                                    protein_dict=protein_dict, protein_res_id_dict=protein_res_id_dict, nci_df=nci_df, ligand_atom_id_dict=ligand_atom_id_dict, cfg_mode=cfg_mode)
            _te, _tr, _va, _te2 = _total_set.get_idx_split()
            test_set = _total_set[:_te]
            train_set = _total_set[_te:_tr]
            val_set = _total_set[_tr:_va]
            test2_set = _total_set[_va:_te2]
                
    if not cfg_timesplit:
        test2_set = None
        train_part, val_part, test_part = cfg_split_strategy[0], cfg_split_strategy[1], cfg_split_strategy[2]
        train_val_split, val_test_split = int(train_part * len(info)), int((train_part+val_part) * len(info))
        qrint(f"{R}Length% split strategy{S}: {train_val_split} - {val_test_split-train_val_split} - {len(info)-val_test_split}. For dataset processing, the results will always be saved.")
        dataset_path = f"{save_files_path}dataset/"
        qrint(f"Processing {R}dataset{S}:")
        if not cfg_use_saved_files:
            os.system(f"rm -r {dataset_path}")
            for _s in ["data"]:
                os.system(f"mkdir -p {dataset_path}{_s}")
        _total_set = MyDataset_VS(f"{dataset_path}data/", df_tr=info.iloc[0:train_val_split], df_te=info.iloc[val_test_split:], df_va=info.iloc[train_val_split:val_test_split], df_te2=info_test2, \
                                    protein_dict=protein_dict, protein_res_id_dict=protein_res_id_dict, nci_df=nci_df, ligand_atom_id_dict=ligand_atom_id_dict, cfg_mode=cfg_mode)
        _te, _tr, _va, _te2 = _total_set.get_idx_split()
        test_set = _total_set[:_te]
        train_set = _total_set[_te:_tr]
        val_set = _total_set[_tr:_va]
        test2_set = _total_set[_va:_te2]
        
    qrint(f"Successfully processed {R}dataset{S}s.")
    qrint(f"-- {R}train set{S} : {len(train_set)}")
    qrint(f"-- {R}val set{S}   : {len(val_set)}")
    qrint(f"-- {R}test set{S}  : {len(test_set)}")
    qrint(f"-- {R}test2 set{S} : {len(test2_set) if test2_set else 0}")

NameError: name 'qrint' is not defined

In [ ]:
    def calculate_aff_score(self, aff_pred, aff_true):
        return torch.sqrt((((aff_pred - aff_true)**2).sum(axis=-1)).mean())